In [0]:
# ! pip install pandas
# ! pip install kashgari
# ! pip install tensorflow==1.15.0



import pandas as pd
import kashgari
import re
import keras

#使用這個目錄下的模型
loaded_model = kashgari.utils.load_model('drive/My Drive/rbt3-model')

#主函式，能替dataframe創造一個新的"tag"欄位，找到該評論的菜名
class get_dishes:

    global loaded_model

    def __init__(self,df):
        self.D = df

    def cut_text(self,text, lenth):
        textArr = re.findall('.{' + str(lenth) + '}', text)
        textArr.append(text[(len(textArr) * lenth):])
        return textArr


    def extract_labels(self,text, ners):
        ner_reg_list = []
        if ners:
            new_ners = []
            for ner in ners:
                new_ners += ner;
            for word, tag in zip([char for char in text], new_ners):
                if tag != 'O':
                    ner_reg_list.append((word, tag))

# 输出模型的NER识别结果
        labels = {}
        if ner_reg_list:
            for i, item in enumerate(ner_reg_list):
                if item[1].startswith('B'):
                    label = ""
                    end = i + 1
                    while end <= len(ner_reg_list) - 1 and ner_reg_list[end][1].startswith('I'):
                        end += 1

                    ner_type = item[1].split('-')[1]
                    if ner_type not in labels.keys():
                        labels[ner_type] = []

                    label += ''.join([item[0] for item in ner_reg_list[i:end]])
                    labels[ner_type].append(label)

        return labels

#建立一個能讓dataframe中的評論欄位,能預測菜單的函式
    def get_name(self,sentence):

        text_1 = self.cut_text(sentence, 100)
        ners = loaded_model.predict([[char for char in text] for text in text_1])
        labels = self.extract_labels(sentence, ners)
        x = labels.get("1","")
        return ",".join(x)

#新增一個tag欄位，放入預測的菜單
    def Get(self):
        self.D["tag"] = self.D["text"].apply(self.get_name)
        return self.D